In [1]:
import os as os
import numpy as np
import scipy.stats as st
import cvxpy as cvx
import pandas as pd
import datetime
import warnings
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import math
warnings.filterwarnings('ignore')


#os.chdir('C:/directindex/ver1')
# os.chdir('C:/Users/admin/Documents/directIndex/baseline')
os.chdir('C:/Users/208017/Documents/directindex/baseline')

universPortName =  'portfolioESG(159)'

constCnt = 100

portName = 'portfolio'+str(constCnt)


#過去指数ファンド情報読み取り
# tickers=pd.read_csv('esgTickers1.csv')
tickers=pd.read_csv('esgTickers3.csv')
tickers=tickers['ticker'].to_list()     #ticker列のデータ取得

#一件目のファンドの過去日付データ読み取り
data = pd.read_csv('./allData/'+tickers[0]+'.csv')
data = data.drop(['Open', 'High','Low','Close','Adj Close','Volume'], axis=1)

dataDate = data.copy()

#過去期間2013-01-02～2018-12-31のファンドの過去データ取得
for ticker in tickers:
    d = pd.read_csv('./allData/'+ticker+'.csv')
    d = d.drop(['Open', 'High','Low','Adj Close','Volume'], axis=1)
    d = d.rename(columns={'Close': ticker}) #列名称をファンド名にリネーム
    
    date1 = d.iat[0,0]
    if date1 == "2013-01-02":
        data = pd.merge(data,d,on='Date')

#日付、年、月、週というデータを取得する
cal = data[['Date']]
cal['Datetime'] = pd.to_datetime(cal['Date'])
cal['year'] = cal['Datetime'].dt.year
cal['month'] = cal['Datetime'].dt.month
cal['week'] = cal['Datetime'].dt.week

cal['mon'] = 1
for index, row in cal.iterrows():
    if index > 0 and cal.iloc[index,3] != cal.iloc[index-1,3]:
        cal.iloc[index,5] = cal.iloc[index-1,5]+1
    else:
        cal.iloc[index,5] = cal.iloc[index-1,5]
        
cal['iweek'] = 1
for index, row in cal.iterrows():
    if index > 0 and cal.iloc[index,4] != cal.iloc[index-1,4]:
        cal.iloc[index,6] = cal.iloc[index-1,6]+1
    else:
        cal.iloc[index,6] = cal.iloc[index-1,6]

#インデックス設定
data = data.set_index('Date')

# 行・列の差分・変化率を取得する
R = data.pct_change()
R = R.fillna(0)

CloseValue = data
# print(CloseValue)

#累積しないリターンを保存
Rd = R
R = R + 1

#累積リータン
R = R.cumprod()

#配列T：縦、N：横
T = Rd.shape[0]
N = Rd.shape[1]

#ベンチマーク情報
Benchmark = pd.read_csv('sp500.csv')
Benchmark = Benchmark.set_index('Date')
Benchmark = Benchmark.drop(['Open', 'High','Low','Adj Close','Volume'], axis=1)

#------------------------------------------------
#シミュレーション
#------------------------------------------------
di = Benchmark.copy() # ポートフォリオの時価総額の推移
Benchmarkd = Benchmark.copy()
Benchmarkd = Benchmarkd.pct_change()
Benchmarkd = Benchmarkd.fillna(0)

did = Benchmark.copy() #日次
did100 = Benchmark.copy() #日次


#最適ウェイト計算の
mon_window = 72 # 6年間72月


w_init = np.ones(N)/N
w_initd = np.ones(N)/N
# w_init = np.ones(N) + 1
w_initq = np.ones(N)
# w_initm = np.ones(N)/N

N100=constCnt
w_init100 = np.ones(N100)/N100
w_initd100 = np.ones(N100)/N100

for i in range(120 - mon_window):
    start = 1
    end = i+mon_window
    
    # 計算対象の開始終了
    startdate = cal.query('mon >= '+str(start)+' and mon <= '+str(end))['Date'].min()
    enddate = cal.query('mon >= '+str(start)+' and mon <= '+str(end))['Date'].max()

    # 日次リターンで最適化
    # 各資産の計算期間分のデータを取得し計算期間初からのリターンを計算
    assetd = Rd[startdate:enddate]
    
    assetd = assetd.values
    
    # ベンチマークの計算期間分のデータを取得し計算期間初からのリターンを計算
    indxd = Benchmarkd[startdate:enddate]
    
    T = len(indxd)
    
    indxd = indxd.values
    indxd = np.squeeze(indxd)

    # 最適化ソルバでウェイトを算出
    weightd = cvx.Variable(N,value=w_initd)
    objd = cvx.Minimize(cvx.sum_squares((assetd@weightd) - indxd))
    mintracerrd = cvx.Problem(objd, [cvx.sum(weightd) == 1.0,weightd >= 0])
    mintracerrd.solve(verbose=False)
    wd = weightd.value

    wd = weightd.value    
    
    wd[wd<0]=0
    wd=wd/wd.sum()
    
#     print(wd)
    
    # 整数最適化
#     print(wd)
#     quantity = cvx.Variable(N, integer=True, value=w_initq)
    
#     assetClose = CloseValue[startdate:enddate]
    
#     assetClose = assetClose.values    
    
#     print(assetClose[-1,:] * quantity.value)
    
#     port_amount = sum(assetClose[-1,:] * quantity.value)
    

    
#     weightd = assetd[-1,:] * quantity.value / port_amount
    
#     print(sum(weightd))
    
#     objd = cvx.Minimize(cvx.sum_squares((assetd@weightd) - indxd))
# #     mintracerrd = cvx.Problem(objd, [cvx.sum(weightd) == 1.0,weightd >= 0])
#     mintracerrd = cvx.Problem(objd, [weightd == assetd[-1,:] * quantity.value / sum(assetClose[-1,:] * quantity.value)])
#     mintracerrd = cvx.Problem(objd, [cvx.sum(weightd) == 1.0,weightd >= 0])
#     objd = cvx.Minimize(cvx.sum_squares(assetd@(assetClose[-1,:] * quantity.value / sum(assetClose[-1,:] * quantity.value)) - indxd))
#     objd1 = cvx.Minimize(cvx.sum_squares(quantity.value))    
#     mintracerrd = cvx.Problem(objd, objd1)
#     mintracerrd.solve(solver='ECOS_BB', verbose=False)
#     print((assetClose[-1,:] * quantity.value / sum(assetClose[-1,:] * quantity.value)))
#     objd1 = cvx.Minimize(cvx.sum_squares((assetClose[-1,:] * quantity.value / sum(assetClose[-1,:] * quantity.value)) - wd))
# #     objd1 = cvx.Minimize(cvx.sum_squares(quantity.value))    
#     mintracerrd1 = cvx.Problem(objd1)
#     mintracerrd1.solve(solver='ECOS_BB', verbose=False)
    
#     print(quantity.value)
    

#     w_initd = wd
    
    #--------------------------------------------------------------------
    # ウェイト上位100のファンドを判断し、再最適化ツルバでウェイトを算出
    wdSorted = sorted(wd, reverse=True)
    max100 = []
    for j in range(constCnt):
        weightValue = wdSorted[j]
        max100 = max100 + [m for m, v in enumerate(wd) if v == weightValue]
            
    tickers100 = []
    tickersList = R.columns
    for n in range(constCnt):
        indexN = max100[n]
        tickers100.append(tickersList[indexN])
    
#     print(tickers100)
    
    Rd100 = Rd.copy()
    R100 = R.copy()
    
    CloseValue100=CloseValue.copy()
    
    for m in range(len(tickersList)):
        colName = tickersList[m]
        n = 0
        for j in range(constCnt):
            if colName == tickers100[j]:
                n = 1
        if(n==0):   
            Rd100 = Rd100.drop(colName,axis=1)
            R100 = R100.drop(colName,axis=1)
            CloseValue100 = CloseValue100.drop(colName,axis=1)

    assetd100 = Rd100[startdate:enddate]    
    assetd100 = assetd100.values
    
    CloseValue100 = CloseValue100[startdate:enddate]
    
    weightd100 = cvx.Variable(N100,value=w_initd100)
#     objd100 = cvx.Minimize(cvx.sum_squares((assetd100@weightd100) - indxd)/np.sqrt(2500))
    objd100 = cvx.Minimize(cvx.sum_squares((assetd100@weightd100) - indxd))    
    mintracerrd100 = cvx.Problem(objd100, [cvx.sum(weightd100) == 1.0,weightd100 >= 0.00001, weightd100/CloseValue100.values[-1,:] >= 0.00004])
#     mintracerrd100 = cvx.Problem(objd100, [cvx.sum(weightd100) == 1.0,weightd100 >= 0])
#     mintracerrd100 = cvx.Problem(objd100, [cvx.sum(weightd100) == 1.0])    
#     mintracerrd100.solve(verbose=True)
    mintracerrd100.solve(solver='ECOS', verbose=False)    
    wd100 = weightd100.value
    
    
    
    
    wd100[wd100<0]=0
    wd100=wd100/wd100.sum()
    w_initd100 = wd100
    
    wd100before = wd100.copy()
    
#     print(wd100)
#     print(CloseValue100.values[-1,:])
    
    
    quan = wd100 / CloseValue100.values[-1,:]
    
    print(math.floor(1/min(quan)))
    
    quan1 = np.floor(25000 * wd100 / CloseValue100.values[-1,:])
    
#     print(quan1)

    print(quan1)

    amount = quan1 * CloseValue100.values[-1,:]
    
    samount= sum(amount)
    
    wd100 = amount / samount
    
#     wd100 = wd100 - 0.0005
    
    print(wd100 - wd100before)
    
#     print((min(quan)))
#     print(min(quan))
    
    
    
#     quan.to_csv('quan'+ i +'.csv')

#     np.savetxt('quan'+ str(i) +'.csv',quan,delimiter=',')

    
#     print(quan)
    
    #----------------------------------------------------------------------
    
    # 計算したウェイトと運用期間の各資産のリターンから運用期間のポートフォリオの時価評価額の比率を計算
    dstart = cal.query('mon >= '+str(end+1)+' and mon <= '+str(end+1))['Date'].min()
    dend = cal.query('mon >= '+str(end+1)+' and mon <= '+str(end+1))['Date'].max()
    dstartm1 = cal.query('mon >= '+str(end)+' and mon <= '+str(end))['Date'].max()

    #Backtest期間
    rtmpd=R[dstartm1:dend]
        
    rtmpd = rtmpd.pct_change()
    rtmpd = rtmpd.dropna()
    rtmpd = rtmpd + 1
    rtmpd = rtmpd.cumprod()

    dd = rtmpd.values@wd
    dd = dd*did.at[dstartm1,'Close']

    dd = np.expand_dims(dd, 1)
    did[dstart:dend]=dd
    
    #--------------------------------------------------------------------
    rtmpd100=R100[dstartm1:dend]
    rtmpd100 = rtmpd100.pct_change()
    rtmpd100 = rtmpd100.dropna()
    rtmpd100 = rtmpd100 + 1
    rtmpd100 = rtmpd100.cumprod()
    dd100 = rtmpd100.values@wd100
    dd100 = dd100*did100.at[dstartm1,'Close']
    dd100 = np.expand_dims(dd100, 1)
    did100[dstart:dend]=dd100
    #----------------------------------------------------------------------

#     print(wd100)
    
import plotly.graph_objects as go
from plotly.subplots import make_subplots
ganpon=100000
fig = make_subplots(rows=2, cols=1)


Benchmark.to_csv('b2.csv')
di.to_csv('d2.csv')

from matplotlib import dates as mdates

heights = [4, 1,1,1]
fig = plt.figure(figsize=(20, 20))
spec = fig.add_gridspec(ncols=1, nrows=4,  height_ratios=heights)
#spec = fig.add_gridspec(ncols=1, nrows=5)
ax = fig.add_subplot(spec[0, 0])

#ベンチマーク
ax.plot(pd.to_datetime(Benchmark.index),Benchmark['Close'],c='k')
#フル460銘柄
ax.plot(pd.to_datetime(did.index),did['Close'],c='b',linestyle="-")
#一部銘柄
ax.plot(pd.to_datetime(did100.index),did100['Close'],c='r',linestyle="-")

print(Benchmark['Close'][-1])
print(did['Close'][-1])

print(did100['Close'][-1])




# ax.legend(labels=['Benchmark S&P500','portfolio460',portName,'portfolio100日次R','portfolio月次R'],fontsize=30)

ax.legend(labels=['Benchmark S&P500',universPortName,portName],fontsize=30)
ax.set_xlabel('date', size=15)
ax.set_ylabel('value', size=15)
ax.set_title('Benchmark vs. Replication Portfolio', fontsize=30)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xticks(rotation=30)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m'))

#
#TE計算
#

portDaily = did.pct_change()
portDaily = portDaily.fillna(0)

portDaily100 = did100.pct_change()
portDaily100 = portDaily100.fillna(0)

ax = fig.add_subplot(spec[1, 0])
ax.plot(pd.to_datetime(Benchmark.index),((Benchmarkd['Close']-portDaily['Close'])**2)**0.5, c='b')
ax.legend(labels=[universPortName],fontsize=20)

ax.set_xlabel('date', size=15)
ax.set_ylabel('tracking error', size=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xticks(rotation=30)

ax = fig.add_subplot(spec[2, 0])
ax.plot(pd.to_datetime(Benchmark.index),((Benchmarkd['Close']-portDaily100['Close'])**2)**0.5, c='r')
ax.legend(labels=[portName],fontsize=20)

ax.set_xlabel('date', size=15)
ax.set_ylabel('tracking error', size=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xticks(rotation=30)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m'))

fig.savefig('tw_esg.png')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/208017/Documents/directindex/baseline'